# Для начала загрузим данные и подготовим все для линейной модели из первого задания

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
from patsy import dmatrices, dmatrix
import numpy as np
from copy import deepcopy
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score, log_loss
from scipy import sparse
%matplotlib inline

In [2]:
df = pd.read_csv(
    '~/Downloads/data.csv'
)

In [3]:
df.head(3)

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1


In [4]:
# Немного уменьшим размер 
df[['clicks']] = df[['clicks']].apply(pd.to_numeric, downcast="unsigned")
df[['zone_id', 'os_id','impressions','country_id','banner_id', 'banner_id0', 'banner_id1']] = df[['zone_id', 'os_id','impressions','country_id','banner_id', 'banner_id0', 'banner_id1']].apply(lambda x: x.astype('category'))
df.drop(columns=['campaign_clicks'], inplace=True)

In [5]:
def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    data["date_time"] =  pd.to_datetime(data["date_time"])
    
    # Чтобы удобнее было делить выборку по времени, когда мы будем работать с разреженной матрицей 
    # и удобно индексироваться по pd.DataFrame уже не получится, отсортируем все
    
    data.sort_values("date_time",inplace=True)  
    data.reset_index(inplace=True, drop=True)  
    data['time'] = data['date_time'].dt.hour.astype(int)*60 + data['date_time'].dt.minute.astype(int)
    data['hour'] = data['date_time'].dt.hour
    data['day'] = data['date_time'].dt.month.astype(int)*30 + data['date_time'].dt.day.astype(int)
    data['month'] = data['date_time'].dt.month
    data['weekday'] = data['date_time'].dt.weekday
    data = data.drop(columns=['date_time'])   # Исходная колонка с временем теперь избыточна
    data[['time','day', 'hour', 'weekday']] = data[['time','day', 'hour', 'weekday']].apply(pd.to_numeric, downcast="unsigned")
    data = data.drop(data.loc[data['day']==271].index)
    data['day'] = data['day'] - (data['day'].min() - 1) # для простоты будем вести счет дней с первого
    data['time_absolute'] = data['day'] *24 *60 + data['time']
    return data 

In [6]:
# columns_to_train = ['date_time','zone_id', 'os_id','country_id','banner_id']
df = feature_engineering(df)

In [7]:
df.head(3)

,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,...,g1,coeff_sum1,impressions,clicks,time,hour,day,month,weekday,time_absolute
1,41,29,1834033519797437404,3,0,29,0.002,0.016386,-4.736584,6,...,0.020875,-4.898257,1,0,0,0,1,9,6,1440
2,1,188,7416450538971744701,2,15,188,0.008,0.014186,-3.811444,11754925,...,0.012351,-2.106896,1,0,0,0,1,9,6,1440
3,17,52,1832228443297591417,2,5,52,0.008,0.013550,-4.317590,41,...,0.067812,-3.739501,1,0,0,0,1,9,6,1440


In [8]:
# Получим индекс для тестовой выборки

df.loc[(df['month'] == 10) & (df['day']==7)]

,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,...,g1,coeff_sum1,impressions,clicks,time,hour,day,month,weekday,time_absolute
13692494,14,1239,6628179337000354250,1,0,1239,0.0460,0.011367,-6.356145,1234,...,0.011368,-6.355855,1,0,0,0,7,10,5,10080
13692495,525,174,2436793977145729294,3,0,174,0.0010,0.063288,-3.107591,104,...,0.041912,-3.091600,1,0,0,0,7,10,5,10080
13692496,14,175,6622864732614000542,2,9,175,0.0070,0.042959,-3.023266,232,...,0.042071,-3.060019,1,0,0,0,7,10,5,10080
13692497,0,76,3615498569651227068,1,3,76,0.0080,0.051014,-4.665202,34,...,0.014532,-4.988883,1,0,0,0,7,10,5,10080
13692498,24,428,684235863208116380,1,10,428,0.0010,0.157651,-3.672826,719,...,0.167411,-3.676580,1,0,0,0,7,10,5,10080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15821467,24,180,1030486855470422958,2,6,180,0.0080,0.038556,-3.791884,403,...,0.026928,-3.749882,1,0,1439,23,7,10,5,11519
15821468,73,92,5392241310084555538,1,0,92,0.0594,0.023820,-4.641030,12584620,...,0.010748,-2.737213,1,0,1439,23,7,10,5,11519
15821469,17,1235,5569517219693927594,4,0,1235,0.0670,0.027977,-5.617197,1240,...,0.027988,-5.616952,1,0,1439,23,7,10,5,11519
15821470,1,2,4888551521096100763,0,0,2,0.0140,0.017499,-3.052530,21,...,0.029316,-3.173799,1,0,1439,23,7,10,5,11519


In [9]:
test_index = 13692494

In [10]:
 # Оставим только частые зоны
freq_zone_ids = set(df['zone_id'].value_counts().loc[lambda x: x > 60000].index)
df['zone_id'] = df['zone_id'].astype(int)
df.loc[~df['zone_id'].isin(freq_zone_ids), 'zone_id'] = 99999

# И баннеры
freq_banners = set(df['banner_id'].value_counts().loc[lambda x: x > 100].index)
df['banner_id'] = df['banner_id'].astype(int)
df.loc[~df['banner_id'].isin(freq_banners), 'banner_id'] = 99999

to_normalize = df[["time", "time_absolute"]].values.astype(float)
normalized_X = normalize(to_normalize)

# 

In [11]:
columns_for_ohe = ['zone_id', 'os_id','country_id','banner_id', 'hour', 'day', 'month', 'weekday']
model_OHE = ColumnTransformer(
                  [('OHE', OneHotEncoder(drop='first', sparse=True, dtype=np.uint8), columns_for_ohe)],
                   remainder = 'passthrough'
                )
model_OHE.fit(df[columns_for_ohe])

ColumnTransformer(remainder='passthrough',
                  transformers=[('OHE',
                                 OneHotEncoder(drop='first',
                                               dtype=<class 'numpy.uint8'>),
                                 ['zone_id', 'os_id', 'country_id', 'banner_id',
                                  'hour', 'day', 'month', 'weekday'])])

In [12]:
X = df[:test_index]
dummified = model_OHE.transform(X[columns_for_ohe])
normalized_X = sparse.csc_matrix(normalized_X[:test_index])
X = sparse.hstack([normalized_X,dummified])
X = X.tocsr()


In [13]:
y = df[:test_index]['clicks']

In [14]:
model = LogisticRegression(penalty='l2',C=1.0,solver='liblinear')
model.fit(X,y)

LogisticRegression(solver='liblinear')

In [32]:
# def deal_with_categorical(data, model_ohe) -> sparse.csc_matrix:
    
#     # Оставим только частые зоны
#     freq_zone_ids = set(data['zone_id'].value_counts().loc[lambda x: x > 60000].index)
#     data['zone_id'] = data['zone_id'].astype(int)
#     data.loc[~data['zone_id'].isin(freq_zone_ids), 'zone_id'] = 99999
    
#     # И баннеры
#     freq_banners = set(data['banner_id'].value_counts().loc[lambda x: x > 100].index)
#     data['banner_id'] = data['banner_id'].astype(int)
#     data.loc[~data['banner_id'].isin(freq_banners), 'banner_id'] = 99999
    
#     # Нормализуем то, что не в [0,1]
#     to_normalize = data[["time", "time_absolute"]].values.astype(float)
#     normalized_X = normalize(to_normalize)
   
    
#     # Сделаем one-hot для всего остального
#     data.drop(columns=["time", "time_absolute"], inplace=True)
#     model_OHE = ColumnTransformer(
#                   [('OHE', OneHotEncoder(drop='first', sparse=True, dtype=np.uint8),data.columns)],
#                    remainder = 'passthrough'
#                 )
#     dummified = model_OHE.fit_transform(data)
#     X = sparse.csc_matrix(normalized_X)
#     X = sparse.hstack([X,dummified])
#     X = X.tocsr()
    
#     return X

In [ ]:
test_predict_proba = model.predict_proba(X_test)
test_logloss = log_loss(y_test, test_predict_proba)
print(f"Best test log loss is {test_logloss}")

# Подготовим данные для подсчета IPS

In [15]:
last_day_0 = df[test_index:][df['banner_id'].astype('int') == df['banner_id0'].astype('int')]

<ipython-input-15-27b4948f7a9b>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  last_day_0 = df[test_index:][df['banner_id'].astype('int') == df['banner_id0'].astype('int')]


In [16]:
# После некоторых проб и ошибок выяснилось, что нужно также выкинуть наны
last_day_0 = last_day_0.dropna(subset = ['g1', 'coeff_sum1'])

In [41]:
last_day_1 = deepcopy(last_day_0)


In [42]:
last_day_0

,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,...,g1,coeff_sum1,impressions,clicks,time,hour,day,month,weekday,time_absolute
13692495,99999,174,2436793977145729294,3,0,174,0.0010,0.063288,-3.107591,104,...,0.041912,-3.091600,1,0,0,0,7,10,5,10080
13692496,14,175,6622864732614000542,2,9,175,0.0070,0.042959,-3.023266,232,...,0.042071,-3.060019,1,0,0,0,7,10,5,10080
13692497,0,76,3615498569651227068,1,3,76,0.0080,0.051014,-4.665202,34,...,0.014532,-4.988883,1,0,0,0,7,10,5,10080
13692498,24,428,684235863208116380,1,10,428,0.0010,0.157651,-3.672826,719,...,0.167411,-3.676580,1,0,0,0,7,10,5,10080
13692499,76,84,2895840198533189080,3,0,84,0.0010,0.034671,-3.015942,149,...,0.080773,-3.265015,1,0,0,0,7,10,5,10080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15821466,20,1236,3530521242255807926,2,0,1236,0.0460,0.009155,-6.085725,13765667,...,0.081177,-2.452728,1,0,1439,23,7,10,5,11519
15821467,24,180,1030486855470422958,2,6,180,0.0080,0.038556,-3.791884,403,...,0.026928,-3.749882,1,0,1439,23,7,10,5,11519
15821468,99999,92,5392241310084555538,1,0,92,0.0594,0.023820,-4.641030,12584620,...,0.010748,-2.737213,1,0,1439,23,7,10,5,11519
15821469,17,1235,5569517219693927594,4,0,1235,0.0670,0.027977,-5.617197,1240,...,0.027988,-5.616952,1,0,1439,23,7,10,5,11519


In [43]:
last_day_1['banner_id1'] = df['banner_id1'].astype(int)
last_day_1.loc[~last_day_1['banner_id1'].isin(freq_banners), 'banner_id1'] = 99999
last_day_1.banner_id = last_day_1.banner_id1

In [44]:
X_test_0 = df[test_index:]
X_test_1 = df[test_index:]
dummified_test_0 = model_OHE.transform(last_day_0[columns_for_ohe])
dummified_test_1 = model_OHE.transform(last_day_1[columns_for_ohe])


to_normalize = last_day_0[["time", "time_absolute"]].values.astype(float)
normalized_X = normalize(to_normalize)
X_test_0 = sparse.hstack([normalized_X,dummified_test_0])
X_test_0 = X_test_0.tocsr()

X_test_1 = sparse.hstack([normalized_X,dummified_test_1])
X_test_1 = X_test_1.tocsr()


In [45]:
X_test_0.shape

(1884334, 1216)

In [46]:
len(last_day_1)

1884334

# Определим нужные функции

In [95]:
from scipy import stats

def calculate_policy(dif_sigma, dif_mean):
    eps = 1e-7 
    return stats.norm.sf((0 - dif_mean) / (dif_sigma), dif_mean, dif_sigma)


def calculate_cips(policy_0, policy_1, df):
    lam = 10
    eps = 1e-7
    return (np.minimum(policy_1/(policy_0 + eps), lam) * df['clicks'].values).sum() / len(df)

# Посчитаем полиси и CIPS

In [96]:
dif_sigma = np.sqrt(last_day_0['g0'] ** 2 + last_day_0['g1'] ** 2)
dif_mean = last_day_0['coeff_sum0'] - last_day_0['coeff_sum1']

In [97]:
policy_0 = calculate_policy(dif_sigma, dif_mean)

In [85]:
last_day0_predictions = model.predict_proba(X_test_0)[:, 1]
last_day1_predictions = model.predict_proba(X_test_1)[:, 1]

In [86]:
from scipy.special import logit

last_day0_logits = logit(last_day0_predictions)
last_day1_logits = logit(last_day1_predictions)

In [87]:
dif_mean_new = last_day0_logits - last_day1_logits

In [98]:
policy_new = calculate_policy(dif_sigma, dif_mean_new)

In [102]:
calculate_cips(policy_0, policy_new, last_day_0)

0.048971130632632316

# Получилась оценка CIPS ~ 0.05